# CryoCon Temperature Controller
## For use with a CryoCon 22C Temperature Controller

## API

Channels can be referenced either by their given name or their letter.
All commands generate a response from the controller, so must only perform queries to keep command and response synched.

**max_temperatrue( loop ):** Returns the maximum set point temperature of the given loop.

**channel_max_temperatrue( loop ):** Returns the maximum set point temperature of the loop controlling the given channel.

**temperature( channel ):** Returns the current temperature of the given channel

**get_channel_loop( channel ):** Returns the loop controlled by the given channel.

**get_range( loop ):** Gets the output range for the loop. Values are [ 'hi', 'mid', 'low' ].

**set_range( loop, range ):** Sets the ouput range for the loop. Range values are [ 'hi', 'mid', 'low' ].

**get_ouput( loop ):** Gets the power output of the loop as a fraction of the full range.

**set_point( channel ):** Returns the set point of the given channel.

**set_temperature( channel, temperature ):** Sets the set point of the controlling loop of the given channel.

**lock( lock ):** Locks or unlocks the front key pad.

**enable():** Engages the temperature controller.

**disable():** Stops the tempreature controller. 

### Properties

**channels:** A dictionary of aliases of the channels.

**channel_names:** A dictionary of given name of the channels.

**loops:** A dictionary of loop:input source pairs.

**max_temps:** A dictionary of maximum set point temperatures for each loop.

**units:** A dictionary of units for each channel.

**enabled:** Returns whether the temperature controller is currently engaged.

In [37]:
# standard imports
import os
import sys
import serial

import logging as log
log.basicConfig( level = log.DEBUG )

# SCPI imports
import scpi_instrument as scpi
import visa

In [51]:
class CryoconController( scpi.SCPI_Instrument ):
    """
    Represents a CryoCon 22C Temperature Controller
    
    Arbitrary SCPI commands can be performed
    treating the hieracrchy of the command as attributes.
    
    To read an property:  inst.p1.p2.p3()
    To call a function:   inst.p1.p2( 'value' )
    To execute a command: inst.p1.p2.p3( '' )
    """
    
    #--- methods ---
    
    def __init__( 
        self, 
        port = None, 
        timeout = 10, 
        baud = 9600
    ):
        """
        Initializes an instance of the controller.
        
        :param port: The port associated to the hardware. [Default: None]
        :param timeout: Communication timeout in seconds. [Default: 10]
        :param baud: The hardware baudrate. [Default: 9600]
        """
        scpi.SCPI_Instrument.__init__( self, port, timeout, '\r\n', '\r\n', '@py' )
    
        self.__channels = {}
        self.__channel_names = {}
        self.__loops = None
        self.__max_temps = None
        self.__units = None
        
        
    def connect( self ):
        super().connect()
        self.__channels = { 
            # canonical channel names
            'a': [ 'cha' ], 
            'b': [ 'chb' ] 
        }
        self.__initialize_channel_names() # user channel names
        
        self.__loops = self.__initialize_loop_sources()
        self.__units = self.__initialize_units()
        self.__max_temps = self.__initialize_max_temps()
        
        # lock keypad, turn remote LED on
        self.lock( True )
        
        
    def disconnect( self ):
        self.lock( False )
        super().disconnect()
        
    
    def __add_name_to_channel( self, channel, name ):
        """
        Adds an assoicated name to the channel.
        
        :param channel: The cahnnel to associate to.
        :param name: The new name of the channel.
        """
        name = name.strip().lower()
        self.channels[ channel ].append( name )

    
    def __initialize_channel_names( self ):
        """
        Gets the user channel name of each channel.
        """
        for chan in self.channels:
            name = self.channel_name( chan )
            self.__channel_names[ chan ] = name
            self.__add_name_to_channel( chan, name )

    
    
    def __initialize_loop_sources( self ):
        """
        Returns the source for each loop.
        
        :returns: A dictionary of loop:channel source pairs.
        """
        loops = { '1': None, '2': None, '3': None, '4': None }
        for loop in loops:
            source = self.query( 'loop {}:source?'.format( loop ) )
            loops[ loop ] = source.lower()
            
        return loops
    
    
    def __initialize_max_temps( self ):
        """
        Get the maximum set point temperature for each loop.
        
        :returns: A dictionary of loop:temperature pairs.
        """
        temps = { '1': None, '2': None, '3': None, '4': None }
        for loop in temps:
            ch = self.get_channel_by_name( self.loops[ loop ] )
            units = self.units[ ch ]
            
            temp = self.max_temperature( loop )
            temps[ loop ] = self.temp2float( temp, ch )
            
        return temps
    
    
    def __initialize_units( self ):
        """
        Gets the units for each input channel.
        
        :returns: A dictionary channel:unit pairs
        """
        units = { 'a': None, 'b': None }
        for ch in units:
            units[ ch ] = self.query( 'input {}:units?'.format( ch ) )
        
        return units
        
    
    def get_channel_by_name( self, name ):
        """
        Returns the canonical channel name of the given user channel name.
        
        :param name: A name describing the desired channel.
        :returns: The canonical channel name ('a' or 'b') or None if no match.
        """
        name = name.strip().lower()
        
        for channel, c_names in self.channels.items():
            if ( name == channel ) or ( name in c_names ):
                return channel
            
        # no match
        return None
    
    
    def channel_name( self, channel ):
        name = self.query( 'input {}:name?'.format( channel ) )
        return name.strip()
    
    
    def get_channel_loop( self, channel ):
        """
        Returns the loop controlled by the given channel.
        
        :param channel: The name of the channel to investigate.
        :returns: The controlled loop.
        """
        channel = self.get_channel_by_name( channel )
        
        for loop in [ '1', '2' ]:
            # loops 1 and 2 are controlled
            source = self.loops[ loop ]
            source = self.get_channel_by_name( source )
            if source == channel:
                return loop
        
        # loop not found
        return None
    
    
    @property
    def channels( self ):
        return self.__channels
    
    
    @property
    def channel_names( self ):
        return self.__channel_names
    
    
    @property
    def loops( self ):
        return self.__loops
    
    
    @property
    def max_temps( self ):
        return self.__max_temps
    
    
    @property
    def units( self ):
        return self.__units
    
    
    @property
    def enabled( self ):
        """
        Returns whether to controller is engaged or not.
        
        :returns: Boolean
        """
        resp = self.control()
        resp = resp.strip().lower()
        
        return ( resp == 'on' )
    
    
    def get_range( self, loop ):
        """
        Returns the range of the given loop.
        
        :param loop: The loop to examine.
        :returns: The range of the loop. 
            Values are [ 'HI', 'MID', 'LOW' ]
        """
        rng = self.query( 'loop {}:range?'.format( loop ) )
        return rng.lower().strip()
        
    
    def set_range( self, loop, rng ):
        """
        Sets the range for the given loop.
        
        :param loop: The loop to modify.
        :param rng: The range to set.
            Values are [ 'hi', 'mid', 'low' ]
        """
        self.query( 'loop {}:range {}'.format( loop, rng ) )
        
        
    def get_output( self, loop ):
        """
        Gets the output power of the given loop as a percent of the full range.
        
        :param loop: The loop to examine.
        :returns: The fraction of the full range output power being applied.
        """
        pwr = self.query( 'loop {}:outpwr?'.format( loop ) )
        pwr = float( pwr )
        pwr /= 100
        
        return pwr
    
    
    def max_temperature( self, loop ):
        """
        Returns the maximum set point temperature for the given loop.
        
        :param loop: The loop to examine.
        :returns: The maximum set point of the loop.
        """
        return self.query( 'loop {}:maxset?'.format( loop ) )
    
    
    def channel_max_temperature( self, channel ):
        """
        Returns the maximum set point temperature for the given channel.
        
        :param channel: The channel to examine.
        :returns: The maximum set point of the channel.
        """
        loop = self.get_channel_loop( channel )
        if loop is None:
            return None
        
        return self.max_temps[ loop ]
            
    
    def temperature( self, channel ):
        """
        Returns the current temperature of the given channel.
        
        :param channel: The channel to set the temperature of.
            Valid values are [ 'a', 'A', 'b', 'B' ] or the channel name.
        :returns: The current temperature of teh channel.
        """
        channel = self.get_channel_by_name( channel )
        temp = self.query( 'input? {}'.format( channel ) )
        
        return float( temp )
        
        
    def set_point( self, channel ):
        """
        Returns the current set point for the given channel name.
        
        :param channel: The channel to examine.
        :returns: The set point of the loop associated to the channel.
        """
        loop = self.get_channel_loop( channel )
            
        if loop is None:
            # No loop found corresponding to channel
            return None
        
        setpt = self.query( 'loop {}:setpt?'.format( loop ) )
        
        return self.temp2float( setpt, channel )
        
    
    def set_temperature( self, channel, temperature ):
        """
        Set the set point temperature of the given channel.
        
        :param channel: The channel to set the temperature of.
        :param temperature: The nwe set point temperature.
        """
        loop = self.get_channel_loop( channel )
        max_temp = self.max_temps[ loop ]
        
        if temperature > max_temp:
            raise RuntimeError( 'Temperature is above maximum.' )
            
        self.query( 'loop {}:setpt {}'.format( loop, temperature ) )
        
          
    def enable( self ):
        """
        Engages control of the controller.
        """
        self.query( 'control' )
        
    def disable( self ):
        """
        Disable the temperature controller.
        """
        self.query( 'stop' )
        
        
    def lock( self, lock ):
        """
        Lock or unlocks teh front keypad.
        
        :param lock: A boolean of whether to lock (True) or unlock (False) teh keypad.
        """
        lock = 'on' if lock else 'off'
        self.query( 'system:lock {}'.format( lock ) )
        
        
    def temp2float( self, temp, channel ):
        """
        Converts a temeprature from a given channel to a float.
        Removes units from number part.
        
        :param temp: The temperature string.
        :param channel: Channel the temperature was acquired from.
        :returns:Float value of the temperature string.
        """
        temp = temp.replace( self.units[ channel ], '' )
        return float( temp )
        

# Work

In [56]:
# cryo = CryoconController( 'COM6' )

DEBUG:pyvisa:Reusing ResourceManager with session 2848879


In [57]:
# cryo.connect()

DEBUG:pyvisa:ASRLCOM6::INSTR - opening ...
DEBUG:pyvisa:ASRLCOM6::INSTR - is open with session 5927610
DEBUG:pyvisa:Serial.write b'*IDN?\r\n'
DEBUG:pyvisa:ASRLCOM6::INSTR - reading 20480 bytes (last status <StatusCode.success_max_count_read: 1073676294>)
DEBUG:pyvisa:Serial.write b'input a:name?\r\n'
DEBUG:pyvisa:ASRLCOM6::INSTR - reading 20480 bytes (last status <StatusCode.success_max_count_read: 1073676294>)
DEBUG:pyvisa:Serial.write b'input b:name?\r\n'
DEBUG:pyvisa:ASRLCOM6::INSTR - reading 20480 bytes (last status <StatusCode.success_max_count_read: 1073676294>)
DEBUG:pyvisa:Serial.write b'loop 1:source?\r\n'
DEBUG:pyvisa:ASRLCOM6::INSTR - reading 20480 bytes (last status <StatusCode.success_max_count_read: 1073676294>)
DEBUG:pyvisa:Serial.write b'loop 2:source?\r\n'
DEBUG:pyvisa:ASRLCOM6::INSTR - reading 20480 bytes (last status <StatusCode.success_max_count_read: 1073676294>)
DEBUG:pyvisa:Serial.write b'loop 3:source?\r\n'
DEBUG:pyvisa:ASRLCOM6::INSTR - reading 20480 bytes (las

In [25]:
# cryo.id

DEBUG:pyvisa:Serial.write b'*IDN?\r\n'
DEBUG:pyvisa:ASRLCOM6::INSTR - reading 20480 bytes (last status <StatusCode.success_max_count_read: 1073676294>)


'Cryo-con,22C,206383,3.24E'

In [62]:
# cryo.disconnect()
# del cryo

DEBUG:pyvisa:Serial.write b'system:lock off\r\n'
DEBUG:pyvisa:ASRLCOM6::INSTR - reading 20480 bytes (last status <StatusCode.success_max_count_read: 1073676294>)
DEBUG:pyvisa:Serial.write b'SYST:LOC \r\n'
DEBUG:pyvisa:ASRLCOM6::INSTR - closing
DEBUG:pyvisa:ASRLCOM6::INSTR - is closed
DEBUG:pyvisa:ASRLCOM6::INSTR - closing
